# Table 1 (and Table 2)
A table of MSE metric and loss values at the final epoch of training for all models. 

In [24]:
from deepuq.analyze.analyze import AggregateCheckpoints
from deepuq.data.data import DataPreparation
from deepuq.train import train
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import copy

In [25]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoints = AggregateCheckpoints()

In [26]:
experiments_df = {
    "DE, 0D, input, low": {
        "model": "DE",
        "size_df": 10000,
        "noise": "low",
        "dim": "0D",
        "injection": "input"
    },
    "DE, 0D, input, medium": {
        "model": "DE",
        "size_df": 10000,
        "noise": "medium",
        "dim": "0D",
        "injection": "input"
    },
    "DE, 0D, input, high": {
        "model": "DE",
        "size_df": 10000,
        "noise": "high",
        "dim": "0D",
        "injection": "input"
    },
    "DE, 0D, output, low": {
        "model": "DE",
        "size_df": 10000,
        "noise": "low",
        "dim": "0D",
        "injection": "output"
    },
    "DE, 0D, output, medium": {
        "model": "DE",
        "size_df": 10000,
        "noise": "medium",
        "dim": "0D",
        "injection": "output"
    },
    "DE, 0D, output, high": {
        "model": "DE",
        "size_df": 10000,
        "noise": "high",
        "dim": "0D",
        "injection": "output"
    },
    "DE, 2D, input, low": {
        "model": "DE",
        "size_df": 500,
        "noise": "low",
        "dim": "2D",
        "injection": "input"
    },
    "DE, 2D, input, medium": {
        "model": "DE",
        "size_df": 500,
        "noise": "medium",
        "dim": "2D",
        "injection": "input"
    },
    "DE, 2D, input, high": {
        "model": "DE",
        "size_df": 500,
        "noise": "high",
        "dim": "2D",
        "injection": "input"
    },
    "DE, 2D, output, low": {
        "model": "DE",
        "size_df": 500,
        "noise": "low",
        "dim": "2D",
        "injection": "output"
    },
    "DE, 2D, output, medium": {
        "model": "DE",
        "size_df": 500,
        "noise": "medium",
        "dim": "2D",
        "injection": "output"
    },
    "DE, 2D, output, high": {
        "model": "DE",
        "size_df": 500,
        "noise": "high",
        "dim": "2D",
        "injection": "output"
    },
    "DER, 0D, input, low": {
        "model": "DER",
        "size_df": 10000,
        "noise": "low",
        "dim": "0D",
        "injection": "input"
    },
    "DER, 0D, input, medium": {
        "model": "DER",
        "size_df": 10000,
        "noise": "medium",
        "dim": "0D",
        "injection": "input"
    },
    "DER, 0D, input, high": {
        "model": "DER",
        "size_df": 10000,
        "noise": "high",
        "dim": "0D",
        "injection": "input"
    },
    "DER, 0D, output, low": {
        "model": "DER",
        "size_df": 10000,
        "noise": "low",
        "dim": "0D",
        "injection": "output"
    },
    "DER, 0D, output, medium": {
        "model": "DER",
        "size_df": 10000,
        "noise": "medium",
        "dim": "0D",
        "injection": "output"
    },
    "DER, 0D, output, high": {
        "model": "DER",
        "size_df": 10000,
        "noise": "high",
        "dim": "0D",
        "injection": "output"
    },
    "DER, 2D, input, low": {
        "model": "DER",
        "size_df": 500,
        "noise": "low",
        "dim": "2D",
        "injection": "input"
    },
    "DER, 2D, input, medium": {
        "model": "DER",
        "size_df": 500,
        "noise": "medium",
        "dim": "2D",
        "injection": "input"
    },
    "DER, 2D, input, high": {
        "model": "DER",
        "size_df": 500,
        "noise": "high",
        "dim": "2D",
        "injection": "input"
    },
    "DER, 2D, output, low": {
        "model": "DER",
        "size_df": 500,
        "noise": "low",
        "dim": "2D",
        "injection": "output"
    },
    "DER, 2D, output, medium": {
        "model": "DER",
        "size_df": 500,
        "noise": "medium",
        "dim": "2D",
        "injection": "output"
    },
    "DER, 2D, output, high": {
        "model": "DER",
        "size_df": 500,
        "noise": "high",
        "dim": "2D",
        "injection": "output"
    },
}

In [27]:
n_models = 16
epoch = 99

inject_type_list = ["output", "input"]
data_dim_list = ["0D", "2D"]
model_type = ["DE", "DER"]
noise_list = ["low", "medium", "high"]

mega_dict = {}

# Nested loops to fill the dictionary
for noise in noise_list:
    mega_dict[noise] = {}  # Create a sub-dictionary for each noise level
    for model in model_type:
        mega_dict[noise][model] = {}  # Create a sub-dictionary for each model type
        for dim in data_dim_list:
            mega_dict[noise][model][dim] = {}  # Create a sub-dictionary for each data dimension
            for inject_type in inject_type_list:
                mega_dict[noise][model][dim][inject_type] = []
mega_dict_MSE_valid = copy.deepcopy(mega_dict)
mega_dict_MSE_train = copy.deepcopy(mega_dict)
mega_dict_loss_valid = copy.deepcopy(mega_dict)
mega_dict_loss_train = copy.deepcopy(mega_dict)

In [28]:
for experiment_name, params in experiments_df.items():
    print(f"Experiment: {experiment_name}")
    for key, value in params.items():
        print(f"  {key}: {value}")
        chk = 0
        if params["model"] == "DER":
            COEFF = 0.01
            chk = checkpoints.load_checkpoint(
                params["model"],
                params["injection"],
                params["dim"],
                params["noise"],
                epoch,
                DEVICE,
                path="../DeepUQResources/checkpoints/",
                COEFF=COEFF,
                load_rs_chk=True,
                rs=42
            )
            MSE = chk["train_mse"]
            loss = chk["train_loss"]
            MSE_valid = chk["valid_mse"]
            loss_valid = chk["valid_loss"]
        elif params["model"] == "DE":
            models_used = 0
            loss = "bnll_loss"
            BETA = 0.5
            MSE = []
            loss = []
            MSE_valid = []
            loss_valid = []
            for m in range(n_models):
                try:
                    chk = checkpoints.load_checkpoint(
                        params["model"],
                        params["injection"],
                        params["dim"],
                        params["noise"],
                        epoch,
                        DEVICE,
                        path="../DeepUQResources/checkpoints/",
                        BETA=BETA,
                        nmodel=m,
                        load_rs_chk=True,
                        rs=int(42+m),
                    )
                except FileNotFoundError:
                    continue
                MSE.append(chk["train_mse"])
                loss.append(chk["train_loss"])
                MSE_valid.append(chk["valid_mse"])
                loss_valid.append(chk["valid_loss"])
        # populate the mega dicts
        mega_dict_MSE_train[params["noise"]][params["model"]][params["dim"]][params["injection"]] = MSE
        mega_dict_loss_train[params["noise"]][params["model"]][params["dim"]][params["injection"]] = loss
        mega_dict_MSE_valid[params["noise"]][params["model"]][params["dim"]][params["injection"]] = MSE_valid
        mega_dict_loss_valid[params["noise"]][params["model"]][params["dim"]][params["injection"]] = loss_valid
        

Experiment: DE, 0D, input, low
  model: DE
  size_df: 10000
  noise: low
  dim: 0D
  injection: input
Experiment: DE, 0D, input, medium
  model: DE
  size_df: 10000
  noise: medium
  dim: 0D
  injection: input
Experiment: DE, 0D, input, high
  model: DE
  size_df: 10000
  noise: high
  dim: 0D
  injection: input
Experiment: DE, 0D, output, low
  model: DE
  size_df: 10000
  noise: low
  dim: 0D
  injection: output
Experiment: DE, 0D, output, medium
  model: DE
  size_df: 10000
  noise: medium
  dim: 0D
  injection: output
Experiment: DE, 0D, output, high
  model: DE
  size_df: 10000
  noise: high
  dim: 0D
  injection: output
Experiment: DE, 2D, input, low
  model: DE
  size_df: 500
  noise: low
  dim: 2D
  injection: input
Experiment: DE, 2D, input, medium
  model: DE
  size_df: 500
  noise: medium
  dim: 2D
  injection: input
Experiment: DE, 2D, input, high
  model: DE
  size_df: 500
  noise: high
  dim: 2D
  injection: input
Experiment: DE, 2D, output, low
  model: DE
  size_df: 500

These dictionaries now hold the MSE metric and loss values for all members of the DE ensemble. I reported the value for the first member of each model ensemble in Table 1 and Table 2.

**Keep in mind that some of these ensemble members do not converge and will have much higher MSE metric and loss values. These members were discarded in the `fig2.ipynb` notebook.

In [29]:
mega_dict_MSE_valid

{'low': {'DE': {'0D': {'output': [0.00011396930494811386,
     0.00011184229515492916,
     0.0001090770383598283,
     0.00012044154573231936,
     0.0001291867665713653,
     0.00010840296454261988,
     0.00018337261280976236,
     0.00011453475599410012,
     0.00012113680713810027,
     0.00014609664503950626,
     0.00011099617404397577,
     0.00015719168004579842,
     0.00011196932609891519,
     0.00012250477448105812,
     0.00011403694225009531],
    'input': [0.00011052814807044342,
     0.00010788365761982277,
     0.00012461947335395962,
     0.0001132916659116745,
     0.0002773019077721983,
     0.00011851421731989831,
     0.00022787979105487466,
     0.00012199419870739803,
     0.00010683852451620623,
     0.00038813770515844226,
     0.00011239930609008297,
     0.00015835512022022158,
     0.00011282356717856601,
     0.00011800085485447198,
     0.00011821897351182997]},
   '2D': {'output': [0.0006166296079754829,
     0.00026294810231775045,
     0.0002812510647

For cases where there's more than one member of the ensemble for the DEs, print out just the first element.

In [35]:
data = mega_dict_loss_valid
for level1_key, level1_value in data.items():
    for level2_key, level2_value in level1_value.items():
        for level3_key, level3_value in level2_value.items():
            for key, value in level3_value.items():
                if isinstance(value, list):
                    print(f"{level1_key} -> {level2_key} -> {level3_key} -> {key}: {value[0]}")

                else:
                    print(f"{level1_key} -> {level2_key} -> {level3_key} -> {key}: {value}")

low -> DE -> 0D -> output: -0.050259169191122055
low -> DE -> 0D -> input: -0.04164571687579155
low -> DE -> 2D -> output: -0.04260421171784401
low -> DE -> 2D -> input: -0.09930209815502167
low -> DER -> 0D -> output: -3.0890681743621826
low -> DER -> 0D -> input: -2.9337875843048096
low -> DER -> 2D -> output: -2.7691240310668945
low -> DER -> 2D -> input: -3.063872814178467
medium -> DE -> 0D -> output: -0.12717702984809875
medium -> DE -> 0D -> input: -0.1199432834982872
medium -> DE -> 2D -> output: -0.13084593415260315
medium -> DE -> 2D -> input: -0.09642498195171356
medium -> DER -> 0D -> output: -1.5028899908065796
medium -> DER -> 0D -> input: -1.458609700202942
medium -> DER -> 2D -> output: -1.4676629304885864
medium -> DER -> 2D -> input: -1.7254090309143066
high -> DE -> 0D -> output: -0.17239521443843842
high -> DE -> 0D -> input: -0.16779257357120514
high -> DE -> 2D -> output: -0.17676381766796112
high -> DE -> 2D -> input: -0.13300170004367828
high -> DER -> 0D -> out